#Structured Dashboard Notebook
This notebook produces the csv dataset for the dashboard.

In [3]:
# import libraries
import pandas as pd 
import numpy as np

In [4]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# # API 

# # An alternative way of getting the most up to date COVID Data
# # drawback: manual effort needed to change column names. 

# import sys
# !{sys.executable} -m pip install sodapy

# from sodapy import Socrata

# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata("data.cms.gov", None)

# # Example authenticated client (needed for non-public datasets):
# # client = Socrata(data.cms.gov,
# #                  MyAppToken,
# #                  userame="user@example.com",
# #                  password="AFakePassword")

# # First 2000 results, returned as JSON from API / converted to Python list of
# # dictionaries by sodapy.
# results = client.get("s2uc-8wxp", limit = 2000000)

# # Convert to pandas DataFrame
# COVID = pd.DataFrame.from_records(results)

In [6]:
# Read all the datasets from google drive

# provider level
provider_info = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/Provider_Info.csv")  

# weekly, provider level
dashboard = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/COVID-19_Nursing_Home_Dataset.csv")

# replacement for hospice
demographic = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/facility_2017 - demographic data complete.csv")

fips = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/fips.csv")

VI = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/COVID-19 Community Vulnerability Index.csv")

CMS_Region = pd.read_excel("/content/drive/My Drive/CapstoneProject/Data/Raw data/CMS Region-State Datatable Key.xlsx")

pr = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/Puerto Rico Zipcode-FIPS Code Key.csv")

#missing geolocation
filledaddress = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/filledaddress.csv") 

tribal_key  = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/CMS Tribal Nursing Home Key.csv")

cluster = pd.read_csv("/content/drive/My Drive/CapstoneProject/Data/Raw data/cluster_results_12_08.csv")



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,82,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#Data Engineering
Data filtering and data type changes are applied to the original datasets. Also added engineered fields.

In [7]:
# tribal key need to change to string type
tribal_key = tribal_key.astype({"Federal Provider Number": 'string'}) 
tribal_key['Federal Provider Number'] = tribal_key['Federal Provider Number'].str.zfill(6)

In [8]:
# filter covid data to post June
week_end_before_June = ['05/24/2020','05/31/2020']
dashboard = dashboard[~dashboard['Week Ending'].isin(week_end_before_June)]  

In [9]:
# if quality check is No, remove all covid data
nullify_list = list(dashboard.columns)

nullify_list.remove("Week Ending")
nullify_list.remove("Federal Provider Number")
nullify_list.remove("Provider Name")
nullify_list.remove("Provider Address")
nullify_list.remove("Provider City")
nullify_list.remove("Provider State")
nullify_list.remove("Provider Zip Code")
nullify_list.remove("Submitted Data")
nullify_list.remove("Passed Quality Assurance Check")
nullify_list.remove("County")
nullify_list.remove("Geolocation")
nullify_list.remove("Total Number of Occupied Beds")

dashboard.loc[dashboard['Passed Quality Assurance Check']=="N",nullify_list] = np.nan

                    

In [10]:
# add column cumulative case fatality ratio since 1/1/2020
dashboard['Resident Cumulative Case Fatality Ratio since 1/1/2020 (COVID-19 Deaths per Confirmed Case + Admissions)'] =  dashboard['Residents Total COVID-19 Deaths'] / (dashboard['Residents Total Confirmed COVID-19']+dashboard['Residents Total Admissions COVID-19'])



In [11]:
# add column cumulative case fatality ratio since 6/21/2020 

temp = dashboard[dashboard['Week Ending']=="06/21/2020"]
temp = temp[['Federal Provider Number','Residents Total COVID-19 Deaths','Residents Total Admissions COVID-19','Residents Total Confirmed COVID-19']]
temp.rename(columns={'Residents Total COVID-19 Deaths': '6/21 death','Residents Total Admissions COVID-19':'6/21 admission',
                'Residents Total Confirmed COVID-19':'6/21 confirm'    }, inplace=True)
dashboard = dashboard.merge(temp, left_on='Federal Provider Number', right_on='Federal Provider Number',how='left')

dashboard['Resident Cumulative Case Fatality Ratio since 6/21/2020 (COVID-19 Deaths per Confirmed Case + Admissions)'] =(dashboard['Residents Total COVID-19 Deaths']-dashboard['6/21 death'])/(dashboard['Residents Total Confirmed COVID-19']+dashboard['Residents Total Admissions COVID-19']-dashboard['6/21 admission']-dashboard['6/21 confirm'])

dashboard.loc[dashboard['Week Ending'].isin(['06/07/2020','06/14/2020','06/21/2020']),['Resident Cumulative Case Fatality Ratio since 6/21/2020 (COVID-19 Deaths per Confirmed Case + Admissions)']] = np.nan

del dashboard['6/21 admission']
del dashboard['6/21 death']
del dashboard['6/21 confirm']

In [12]:
# Residents Weekly Confirmed COVID-19 Cases + Admissions 
# "Residents Weekly Confirmed COVID-19" + "Residents Weekly Admissions COVID-19"
dashboard['Residents Weekly Confirmed COVID-19 Cases + Admissions'] = dashboard["Residents Weekly Confirmed COVID-19"] + dashboard["Residents Weekly Admissions COVID-19"]
dashboard['Resident Weekly Infection Rate'] = dashboard['Residents Weekly Confirmed COVID-19']/dashboard['Total Number of Occupied Beds']


#Join datasets to the COVID dataset
FIPS Code is joined to the covid dataset


In [13]:
# FIPS code 
# FIPS 5-digit Code, state 2 digit, county 2 digit, 
# join on zip code 
fips_columns = ['zipcode_char','zip_state_fips','zip_county_fips','FIPS_Code','State']
fips = fips[fips_columns]
dashboard = dashboard.merge(fips, left_on='Provider Zip Code', right_on='zipcode_char',how='left')

In [14]:
# Puerto Rico dataset
pr = pr[['ZIPCODE','STCOUNTYFP']]
dashboard = dashboard.merge(pr, left_on='Provider Zip Code', right_on='ZIPCODE',how='left')
del dashboard['ZIPCODE']


In [15]:
dashboard['FIPS_Code'] = dashboard['FIPS_Code'].fillna(dashboard["STCOUNTYFP"])


In [16]:
# Vunerability index
vi_columns = ['FIPS (5-digit)','CCVI SCORE\nHigher = More Vulnerable']
VI = VI[vi_columns]
dashboard = dashboard.merge(VI, left_on='FIPS_Code', right_on='FIPS (5-digit)',how='left')

In [17]:
# CMS region
dashboard = dashboard.merge(CMS_Region, left_on='State', right_on='State',how='left')

In [18]:
#formatting Federal Provider # and Provider Address

dashboard['Federal Provider Number'] = dashboard['Federal Provider Number'].map(str)
dashboard['Provider Address'] = dashboard['Provider Address'].str.strip()
dashboard['Provider Address'] = dashboard['Provider Address'].replace('\s+', ' ', regex = True)

#creating a new column to merge on
dashboard['Full_Address'] = dashboard['Provider Address'].map(str) + ',' + dashboard['Provider City'].map(str) + ',' + dashboard['Provider State'].map(str) + ',' + dashboard['Provider Zip Code'].map(str)

#renaming column to match COVID-19
filledaddress.rename(columns = {"input_string": "Full_Address"}, inplace=True)

#Removing unnecessary column
filledaddress.drop('Unnamed: 0', inplace = True, axis = 1)

#Removing Duplicates
filledaddress.drop_duplicates(subset=['Full_Address'], inplace= True)

 #formatting to match COVID-19 dataset
filledaddress['geoinput'] = 'POINT (' + filledaddress['longitude'].map(str) + ' ' + filledaddress['latitude'].map(str) + ')'

#Merging 2 datasets to fix missing geolocation
dashboard = dashboard.merge(filledaddress, on= 'Full_Address', how = 'left', indicator=True)
dashboard['Geolocation'].fillna(dashboard['geoinput'], inplace=True)

In [19]:
# extract longitude and latitude from geolocation
dashboard[['Facility Longitude', 'Facility Latitude']]= dashboard['Geolocation'].str.split(r'\s|\(|\)',expand = True).iloc[:,2:4]

In [20]:
# extract month, day, year from week ending
dashboard[['MM', 'DD','YYYY']] = dashboard['Week Ending'].str.split("/",expand = True)

In [21]:
# all the field that need to add binary variable
binary_list = ['Submitted Data','Passed Quality Assurance Check','Able to Test or Obtain Resources to Test All Current Residents Within Next 7 Days','Reason for Not Testing Residents - Lack of PPE for Personnel ','Reason for Not Testing Residents - Lack of Supplies',
'Reason for Not Testing Residents  - Lack of Access to Laboratory','Reason for Not Testing Residents - Lack of Access to Trained Personnel ','Reason for Not Testing Residents  - Uncertainty About Reimbursement',
'Reason for Not Testing Residents  - Other','During Past Two Weeks Average Time to Receive Resident Test Results','Has Facility Performed Resident Tests Since Last Report','Tested Residents with New Signs or Symptoms',
'Tested Asymptomatic Residents in a Unit or Section After a New Case','Tested Asymptomatic Residents Facility-Wide After a New Case','Tested Asymptomatic Residents Without Known Exposure as Surveillance',
'Tested Another Subgroup of Residents','Able to Test or Obtain Resources to Test All Staff and/or Personnel Within Next 7 Days','Reason for Not Testing Staff and/or Personnel - Lack of PPE for Personnel ',
'Reason for Not Testing Staff and/or Personnel - Lack of Supplies','Reason for Not Testing Staff and/or Personnel - Lack of Access to Laboratory','Reason for Not Testing Staff and/or Personnel  - Lack of Access to Trained Personnel ',
'Reason for Not Testing Staff and/or Personnel - Uncertainty About Reimbursement','Reason for Not Testing Staff and/or Personnel - Other','During Past Two Weeks Average Time to Receive Staff and/or Personnel Test Results',
'Has Facility Performed Staff and/or Personnel Tests Since Last Report','Tested Staff and/or Personnel with New Signs or Symptoms','Tested Asymptomatic Staff and/or Personnel in a Unit or Section After a New Case',
'Tested Asymptomatic Staff and/or Personnel Facility-Wide After a New Case','Tested Asymptomatic Staff and/or Personnel Without Known Exposure as Surveillance','Tested Another Subgroup of Staff and/or Personnel',
'In-House Point-of-Care Test Machine','COVID-19 Point-of-Care Tests Performed on Residents Since Last Report','COVID-19 Point-of-Care Tests Performed on Staff and/or Personnel Since Last Report','Enough Supplies to Test All Staff and/or Personnel Using Point-of-Care Test Machine',
'Shortage of Nursing Staff','Shortage of Clinical Staff','Shortage of Aides','Shortage of Other Staff','Any Current Supply of N95 Masks','One-Week Supply of N95 Masks','Any Current Supply of Surgical Masks',
'One-Week Supply of Surgical Masks','Any Current Supply of Eye Protection','One-Week Supply of Eye Protection','Any Current Supply of Gowns','One-Week Supply of Gowns','Any Current Supply of Gloves','One-Week Supply of Gloves',
'Any Current Supply of Hand Sanitizer','One-Week Supply of Hand Sanitizer','Ventilator Dependent Unit','Number of Ventilators in Facility','Number of Ventilators in Use for COVID-19','Any Current Supply of Ventilator Supplies',
'One-Week Supply of Ventilator Supplies']

In [22]:
# binary variable added, change Y/N to 1/0
for field in binary_list:
    binary_name = field+" "+"binary"
    dashboard[binary_name] = dashboard[field].map({'Y': 1, 'N': 0})

In [23]:
# delete unused column in dashboard
del_columns = ['zipcode_char','State','FIPS (5-digit)','Resident Access to Testing in Facility','Laboratory Type Is State Health Dept',
              'Laboratory Type Is Private Lab','Laboratory Type Is Other','Geolocation','Three or More Confirmed COVID-19 Cases This Week',
              'Initial Confirmed COVID-19 Case This Week']
for del_column in del_columns:
    if del_column in dashboard.columns:
        del dashboard[del_column]

In [24]:
# rename columns
dashboard.rename(columns={'Residents Total Admissions COVID-19': 'Residents Total Admissions COVID-19 since January 1, 2020',
                          'Residents Weekly Confirmed COVID-19':'Residents Weekly Confirmed COVID-19 Cases','Residents Total Confirmed COVID-19':'Residents Total Confirmed COVID-19 Cases since January 1, 2020',
                          'Residents Weekly Suspected COVID-19':'Residents Weekly Suspected COVID-19 Cases','Residents Total Suspected COVID-19':'Residents Total Suspected COVID-19 Cases',
                          'Residents Total All Deaths':'Residents Total All Deaths since January 1, 2020','Residents Total COVID-19 Deaths':'Residents Total COVID-19 Deaths since January 1, 2020',
                          'Total Number of Occupied Beds':'Number of Residents in Facility','Total Resident Confirmed COVID-19 Cases Per 1,000 Residents':'Total Resident Confirmed COVID-19 Cases Per 1,000 Residents since 1/1/2020',
                          'Total Resident COVID-19 Deaths Per 1,000 Residents':'Total Resident COVID-19 Deaths Per 1,000 Residents since 1/1/2020','Total Residents COVID-19 Deaths as a Percentage of Confirmed COVID-19 Cases':'Total Residents COVID-19 Deaths as a Percentage of Confirmed COVID-19 Cases since 1/1/2020',
                          'County':'Provider County','''CCVI SCORE
Higher = More Vulnerable''':'County Social Vulnerability Index Score (0-1)'}, inplace=True)

Join demographic dataset

In [25]:

demographic = demographic.replace('LNE',np.nan)
demographic = demographic.replace('.',np.nan)

In [26]:
# change percentage number format
demographic['pctblack_2011p'] =demographic['pctblack_2011p'].astype({ "pctblack_2011p":"float"})/100
demographic['pcthisp_2011p'] =demographic['pcthisp_2011p'].astype({ "pcthisp_2011p":"float"})/100
demographic['pctwhite_2011p'] =demographic['pctwhite_2011p'].astype({ "pctwhite_2011p":"float"})/100
demographic['pctfem'] =demographic['pctfem'].astype({ "pctfem":"float"})/100
demographic['pctlocare_2011p'] =demographic['pctlocare_2011p'].astype({ "pctlocare_2011p":"float"})/100
demographic['pctunder65'] =demographic['pctunder65'].astype({ "pctunder65":"float"})/100
demographic['pctlowcfs'] =demographic['pctlowcfs'].astype({ "pctlowcfs":"float"})/100
demographic['pctmidcfs'] =demographic['pctmidcfs'].astype({ "pctmidcfs":"float"})/100
demographic['pcthighcfs'] =demographic['pcthighcfs'].astype({ "pcthighcfs":"float"})/100
demographic['pctbedft_2011p'] =demographic['pctbedft_2011p'].astype({ "pctbedft_2011p":"float"})/100
demographic['pctwalking'] =demographic['pctwalking'].astype({ "pctwalking":"float"})/100
demographic['pctincont_bladr_2011p'] =demographic['pctincont_bladr_2011p'].astype({ "pctincont_bladr_2011p":"float"})/100
demographic['pctincont_bowel_2011p'] =demographic['pctincont_bowel_2011p'].astype({ "pctincont_bowel_2011p":"float"})/100
demographic['pctcath_2011p'] =demographic['pctcath_2011p'].astype({ "pctcath_2011p":"float"})/100
demographic['pctchf'] =demographic['pctchf'].astype({ "pctchf":"float"})/100
demographic['pcthyper'] =demographic['pcthyper'].astype({ "pcthyper":"float"})/100
demographic['pctschiz_bipol'] =demographic['pctschiz_bipol'].astype({ "pctschiz_bipol":"float"})/100
demographic['pctvent_2011p'] =demographic['pctvent_2011p'].astype({ "pctvent_2011p":"float"})/100
demographic['pctuti'] =demographic['pctuti'].astype({ "pctuti":"float"})/100
demographic['pctfall30_2011p'] =demographic['pctfall30_2011p'].astype({ "pctfall30_2011p":"float"})/100
demographic['pctobese'] =demographic['pctobese'].astype({ "pctobese":"float"})/100


In [27]:
# these are the columns needed from demographic dataset
selected_demo_columns = ['PROV1680','alzunit','anyunit','paymcaid','paymcare','multifac','restrain','acuindex2','anymdex','rn2nrs','avgage','avgadl_2011p',
'avgrugcmi_2011p','pctlocare_2011p','pctfem','pctblack_2011p','pcthisp_2011p','pctwhite_2011p','pctunder65','pctlowcfs',
'pctmidcfs','pcthighcfs','pctbedft_2011p','pctwalking','pctincont_bladr_2011p','pctincont_bowel_2011p','pctcath_2011p','pctchf','pcthyper','pctschiz_bipol',
'pctvent_2011p','pctuti','pctfall30_2011p','pctobese','NHCADL_2011p','adj_rehosprate','adj_successfuldc','adj_medianlos']

In [28]:
demographic = demographic[selected_demo_columns]

In [29]:
# rename columns
demographic.rename(columns={'pctfem': 'Percent Female Residents in 2017','pctblack_2011p':'Percent Non-Hispanic Black Residents in 2017',
                           'pcthisp_2011p':'2017 Resident Percent Hispanic',
                           'pctwhite_2011p':'2017 Resident Percent White Non-Hispanic','avgage':'2017 Resident Average Age','avgadl_2011p':'2017 Resident Average Activites of Daily Living Score',
                         'avgrugcmi_2011p':'2017 Average Resource Utilizationb Group Nursing Case Mix Index (Intensity of Care)','pctlocare_2011p':'2017 Resident % Low Care',
                       'pctunder65':'2017 Resident Percent Under Age 65','pctlowcfs':'2017 Resident Percent Low Cognitive Impairment','pctmidcfs':'2017 Resident Percent Moderate Cognitive Impairment',
                     'pcthighcfs':'2017 Resident Percent  Severe Cognitive Impairment','pctbedft_2011p':'2017 Resident Percent on Bedfast','pctwalking':'2017 Resident Percent Able to Walk in Corridor Independently',
                   'pctincont_bladr_2011p':'2017 Resident Percent Bladder Incontinent','pctincont_bowel_2011p':'2017 Resident Percent Bower Incontinent','pctcath_2011p':'2017 Resident Percent with Catheter',
                 'pctchf':'2017 Resident Percent with Congestive Heart Failure','pcthyper':'2017 Resident Percent with Hypertension','pctschiz_bipol':'2017 Resident Percent with Schizophrenia or Bi-polar Disorder',
               'pctvent_2011p':'2017 Resident Percent on Ventilator','pctuti':'2017 Resident Percent with UTI','pctfall30_2011p':'2017 Resident Percent Fall Prevalence',
             'pctobese':'2017 Resident Percent Obese','NHCADL_2011p':'2017 Residents  Percent Long Stay with Activities of Daily Living Decline','adj_rehosprate':'2017 Re-Hospitalization Rate',
           'adj_successfuldc':'2017 Successful Discharge Rate','adj_medianlos':'2017 Average Length of Stay','acuindex2':'2017 Average Acuity Index'}, inplace=True)

In [30]:
# left join on the coivd dataset
dashboard = dashboard.merge(demographic, left_on='Federal Provider Number', right_on='PROV1680',how='left')
del dashboard['PROV1680']

Join provider info dataset to the COVID dataset

In [31]:
# all the columns needed in the provide info dataset
provider_info_columns = ['Federal Provider Number','Ownership Type','Continuing Care Retirement Community','Special Focus Status','Abuse Icon','Most Recent Health Inspection More Than 2 Years Ago',
'Provider Changed Ownership in Last 12 Months','With a Resident and Family Council','Overall Rating','QM Rating','Staffing Rating','RN Staffing Rating','Reported Licensed Staffing Hours per Resident per Day',
'Reported Physical Therapist Staffing Hours per Resident Per Day','Adjusted Total Nurse Staffing Hours per Resident per Day','Total Weighted Health Survey Score','Number of Facility Reported Incidents',
'Number of Substantiated Complaints','Number of Fines','Total Amount of Fines in Dollars','Number of Payment Denials','Total Number of Penalties']

In [32]:
provider_info = provider_info[provider_info_columns]

In [33]:
# rename columns
provider_info.rename(columns={'Federal Provider Number': 'Federal Provider Number provider'}, inplace=True)

In [34]:
# left join provider info to the covid datset
dashboard = dashboard.merge(provider_info, left_on='Federal Provider Number', right_on='Federal Provider Number provider',how='left')
del dashboard['Federal Provider Number provider']

Join tribal key to the COVID dataset

In [35]:
# select columns
tribal_key = tribal_key[['Federal Provider Number','Tribal Nursing Home','Percent American Indian or Alaska Native Beneficiaries']]

In [36]:
# left join tribal key to the COVID dataset
dashboard = dashboard.merge(tribal_key, left_on='Federal Provider Number', right_on='Federal Provider Number',how='left')
del dashboard['latitude']
del dashboard['longitude']

Join TRAJ Model result to the COVID dataset

In [37]:

cluster = cluster.astype({"federalprovidernumber": 'string'}) 
cluster['federalprovidernumber'] = cluster['federalprovidernumber'].str.zfill(6)
dashboard = dashboard.merge(cluster, left_on='Federal Provider Number', right_on='federalprovidernumber',how='left')

In [38]:
del dashboard['federalprovidernumber']

#Produce Dashboard output CSV, save to Drive

In [39]:
# change the date of the file when running this.
# dashboard.to_csv('/content/drive/My Drive/CapstoneProject/Data/Processed Output/dashboard_12_10.csv', index=False)